<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
LightGBM for Regression 
</p>

In [2]:
# Libraries for data manipulation and visualization
import numpy as np                               # For numerical operations
import pandas as pd                              # For data manipulation
import matplotlib.pyplot as plt                  # For plotting
import seaborn as sns                            # For advanced data visualization

# Libraries for model building and evaluation
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (                     # For model evaluation metrics
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error, 
    median_absolute_error
)

# LightGBM library
import lightgbm as lgb                            # For LightGBM Regressor

# ================================
# Set Seed for Reproducibility
# ================================
RANDOM_STATE = 50
np.random.seed(RANDOM_STATE)

# Suppress specific FutureWarning about 'force_all_finite'
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*'force_all_finite'.*")

In [3]:
# Define the path to the dataset
dataset_path = '../../all_data_files/cleaned_dataset_per_device.csv'

# Load the dataset
try:
    df = pd.read_csv(dataset_path)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"File not found at the specified path: {dataset_path}")
    import sys
    sys.exit()

# Display dataset information
print("\nDataset Information:")
df.info()

print("\nFirst Five Rows of the Dataset:")
display(df.head())

Dataset loaded successfully.

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415654 entries, 0 to 1415653
Data columns (total 20 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   time         1415654 non-null  object 
 1   device_id    1415654 non-null  object 
 2   co2          1415654 non-null  float64
 3   humidity     1415654 non-null  float64
 4   pm25         1415654 non-null  float64
 5   pressure     1415654 non-null  float64
 6   temperature  1415654 non-null  float64
 7   rssi         1415654 non-null  float64
 8   snr          1415654 non-null  float64
 9   SF           1415654 non-null  int64  
 10  frequency    1415654 non-null  float64
 11  f_count      1415654 non-null  float64
 12  p_count      1415654 non-null  float64
 13  toa          1415654 non-null  float64
 14  distance     1415654 non-null  int64  
 15  c_walls      1415654 non-null  int64  
 16  w_walls      1415654 non-null  int64  


,time,device_id,co2,humidity,pm25,pressure,temperature,rssi,snr,SF,frequency,f_count,p_count,toa,distance,c_walls,w_walls,exp_pl,n_power,esp
0,2024-09-26 11:01:52.383162+00:00,ED3,645.0,54.18,0.32,300.48,23.87,-74.0,12.5,9,867.7,95.0,105.0,0.246784,18,1,2,91.26,-86.737602,-74.237602
1,2024-09-26 11:02:08.387851+00:00,ED0,539.0,49.34,0.39,299.69,24.57,-48.0,8.5,9,867.5,82.0,109.0,0.246784,10,0,0,65.26,-57.073822,-48.573822
2,2024-09-26 11:02:14.517123+00:00,ED1,471.0,45.29,0.32,300.80,25.64,-56.0,12.5,9,867.1,81.0,107.0,0.246784,8,1,0,73.26,-68.737602,-56.237602
3,2024-09-26 11:02:24.466271+00:00,ED2,659.0,49.09,0.44,301.18,24.57,-70.0,9.0,8,868.5,77.0,113.0,0.133632,23,0,2,87.26,-79.514969,-70.514969
4,2024-09-26 11:02:26.872032+00:00,ED4,423.0,38.86,0.13,301.69,27.71,-90.0,11.5,8,867.9,3597.0,4224.0,0.133632,37,0,5,107.26,-101.797062,-90.297062


In [4]:
# Define feature columns and target
feature_columns = [
    'distance', 'frequency', 'c_walls', 'w_walls', 
    'co2', 'humidity', 'pm25', 'pressure', 
    'temperature', 'snr'
]
target_column = 'exp_pl'

# Verify that all required columns exist
missing_columns = set(feature_columns + [target_column]) - set(df.columns)
if missing_columns:
    raise ValueError(f"The following required columns are missing in the dataset: {missing_columns}")

# Extract features and target
all_features = df[feature_columns].values
PL_all = df[target_column].values

# Perform train-test split (80-20 split)
X_train_all, X_test_all, PL_train_all, PL_test_all = train_test_split(
    all_features, PL_all, test_size=0.2, random_state=RANDOM_STATE
)

print("Train-test split completed.")

Train-test split completed.


In [5]:
# Define a single set of hyperparameters manually
hyperparameters = {
    'max_depth': 80,                              # Maximum depth of the tree
    'num_leaves': 20,                             # Set num_leaves to 2^max_depth - 1 for optimal performance
    'learning_rate': 0.05,                       # Step size shrinkage
    'n_estimators': 300,                         # Number of boosting iterations
    'subsample': 0.9,                            # Subsample ratio of the training instances
    'colsample_bytree': 0.2,                     # Subsample ratio of columns when constructing each tree
    'min_child_weight': 10,                       # Minimum sum of instance weight (hessian) needed in a child
    'reg_alpha': 0.5,                            # L1 regularization term on weights
    'reg_lambda': 0.5,                           # L2 regularization term on weights
    'objective': 'regression',                   # Regression task
    'verbosity': -1,                             # Suppress LightGBM warnings
    'random_state': 50,                          # Random seed
    'n_jobs': -1                                 # Use all available cores
}

print("\nTraining LightGBM model with hyperparameters.....\n")

# Initialize the LightGBM Regressor with the specified hyperparameters
lgb_model = lgb.LGBMRegressor(**hyperparameters)

# Train the model
lgb_model.fit(X_train_all, PL_train_all)

print("Model training completed....!\n")


Training LightGBM model with hyperparameters.....

Model training completed....!



In [6]:
# Initialize a list to store evaluation metrics for the model
evaluation_metrics = []

print("\nEvaluating the trained LightGBM model...")

# Make predictions on the training set
PL_train_pred = lgb_model.predict(X_train_all)
# Make predictions on the test set
PL_test_pred = lgb_model.predict(X_test_all)

# Calculate metrics
train_mse = mean_squared_error(PL_train_all, PL_train_pred)
test_mse = mean_squared_error(PL_test_all, PL_test_pred)
train_r2 = r2_score(PL_train_all, PL_train_pred)
test_r2 = r2_score(PL_test_all, PL_test_pred)
test_rmse = np.sqrt(test_mse)
test_mape = mean_absolute_percentage_error(PL_test_all, PL_test_pred)
test_median_ae = median_absolute_error(PL_test_all, PL_test_pred)

# Append metrics to the list
evaluation_metrics.append({
    'Model': 'LightGBM',
    'Training Loss (MSE)': train_mse,
    'Test Loss (MSE)': test_mse,
    'Test RMSE': test_rmse,
    'R² Score': test_r2,
    'Test MAPE (%)': test_mape * 100,
    'Test Median AE': test_median_ae
})

# Convert the metrics list to a DataFrame
evaluation_df = pd.DataFrame(evaluation_metrics)

print(f"\nModel Evaluation Metrics:")
# Display the transposed DataFrame
evaluation_df = evaluation_df.T[1:].reset_index(names=["Metric"]).rename(columns={0: "Value"})
display(evaluation_df)


Evaluating the trained LightGBM model...

Model Evaluation Metrics:


,Metric,Value
0,Training Loss (MSE),30.457842
1,Test Loss (MSE),30.54425
2,Test RMSE,5.526685
3,R² Score,0.915823
4,Test MAPE (%),4.662555
5,Test Median AE,3.08727


In [8]:
# Define the number of folds for cross-validation
n_folds = 5

# Initialize KFold with consistent RANDOM_STATE
kf = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)

# Initialize a list to store evaluation metrics for each fold
cv_results = []

print("\nPerforming K-Fold Cross-Validation...\n")

# Iterate over each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_all), 1):

    # Split the data for the current fold
    X_train_fold, X_val_fold = X_train_all[train_idx], X_train_all[val_idx]
    PL_train_fold, PL_val_fold = PL_train_all[train_idx], PL_train_all[val_idx]

    # Initialize the LightGBM Regressor with the specified hyperparameters
    lgbm_cv = lgb.LGBMRegressor(**hyperparameters)

    # Train the model on the current fold
    lgbm_cv.fit(X_train_fold, PL_train_fold)

    # Make predictions on the validation set
    PL_val_pred = lgbm_cv.predict(X_val_fold)

    # Calculate evaluation metrics
    val_mse = mean_squared_error(PL_val_fold, PL_val_pred)
    val_rmse = np.sqrt(val_mse)
    val_r2 = r2_score(PL_val_fold, PL_val_pred)
    val_mape = mean_absolute_percentage_error(PL_val_fold, PL_val_pred)
    val_median_ae = median_absolute_error(PL_val_fold, PL_val_pred)

    # Append metrics to the cv_results list
    cv_results.append({
        'Fold': fold,
        'Validation Loss (MSE)': round(val_mse, 4),
        'Validation RMSE': round(val_rmse, 4),
        'R² Score': round(val_r2, 4),
        'Validation MAPE (%)': round(val_mape * 100, 2),
        'Validation Median AE': round(val_median_ae, 4)
    })

    # Print metrics for the current fold
    print(f" Fold {fold} - MSE: {val_mse:.4f}, RMSE: {val_rmse:.4f}, R²: {val_r2:.4f}, "
          f"MAPE: {val_mape*100:.2f}%, Median AE: {val_median_ae:.4f}\n")

# Convert the list of dictionaries to a DataFrame for better visualization
cv_results_df = pd.DataFrame(cv_results)

print("K-Fold Cross-Validation Results:")
display(cv_results_df)

# Summary statistics
cv_summary = cv_results_df.agg(['mean', 'std']).round(4).reset_index()
cv_summary.rename(columns={'index': 'Metric'}, inplace=True)

# Transposing the summary
cv_summary_transposed = cv_summary.set_index('Metric').T

print("\nCross-Validation Summary:")
display(cv_summary_transposed)


Performing K-Fold Cross-Validation...

 Fold 1 - MSE: 30.6826, RMSE: 5.5392, R²: 0.9160, MAPE: 4.68%, Median AE: 3.0941

 Fold 2 - MSE: 30.6330, RMSE: 5.5347, R²: 0.9162, MAPE: 4.68%, Median AE: 3.1029

 Fold 3 - MSE: 30.2497, RMSE: 5.5000, R²: 0.9168, MAPE: 4.65%, Median AE: 3.0725

 Fold 4 - MSE: 30.4923, RMSE: 5.5220, R²: 0.9163, MAPE: 4.66%, Median AE: 3.0868

 Fold 5 - MSE: 30.6487, RMSE: 5.5361, R²: 0.9158, MAPE: 4.67%, Median AE: 3.0973

K-Fold Cross-Validation Results:


,Fold,Validation Loss (MSE),Validation RMSE,R² Score,Validation MAPE (%),Validation Median AE
0,1,30.6826,5.5392,0.9160,4.68,3.0941
1,2,30.6330,5.5347,0.9162,4.68,3.1029
2,3,30.2497,5.5000,0.9168,4.65,3.0725
3,4,30.4923,5.5220,0.9163,4.66,3.0868
4,5,30.6487,5.5361,0.9158,4.67,3.0973



Cross-Validation Summary:


Metric,mean,std
Fold,3.0000,1.5811
Validation Loss (MSE),30.5413,0.1784
Validation RMSE,5.5264,0.0161
R² Score,0.9162,0.0004
Validation MAPE (%),4.6680,0.0130
Validation Median AE,3.0907,0.0117
